In [1]:
#files failed processing: 300to470

import awkward as ak
import numpy as np
import time
import coffea

print(coffea.__version__)
from coffea import util
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from plugins import handleData

redirector = 'root://xcache/'
files = handleData('fileset_QCD.json', redirector, year = 2018, testing = True, data = False)
events = NanoEventsFactory.from_root('root://xcache//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/6BEB9A7B-150C-7440-96D8-17A3D87F3225.root', schemaclass=NanoAODSchema).events()
print(events.fields)

import uproot
### uproot notation
qcd_mc = uproot.open(list(files.values())[-1][0])
qcd_mc.keys()
uprootevents = qcd_mc['Events']
# uprootevents.keys('HLT_PFJet*')
uprootevents.keys('*Jet*')
# uprootevents.show()

0.7.18
['HLT', 'PuppiMET', 'genWeight', 'Pileup', 'SoftActivityJetNjets10', 'L1', 'luminosityBlock', 'OtherPV', 'TrigObj', 'HTXS', 'TkMET', 'GenVisTau', 'Generator', 'IsoTrack', 'SubGenJetAK8', 'SoftActivityJetHT', 'Muon', 'GenDressedLepton', 'SoftActivityJetHT10', 'genTtbarId', 'CorrT1METJet', 'GenIsolatedPhoton', 'DeepMETResponseTune', 'SoftActivityJetHT2', 'HLTriggerFinalPath', 'Jet', 'HLTriggerFirstPath', 'CaloMET', 'RawPuppiMET', 'SV', 'Photon', 'GenJetAK8', 'Tau', 'PSWeight', 'GenMET', 'SoftActivityJetNjets2', 'Flag', 'GenVtx', 'SoftActivityJetHT5', 'LowPtElectron', 'RawMET', 'ChsMET', 'fixedGridRhoFastjetCentralNeutral', 'event', 'btagWeight', 'run', 'SoftActivityJet', 'MET', 'PV', 'DeepMETResolutionTune', 'L1PreFiringWeight', 'L1Reco', 'L1simulation', 'FatJet', 'SoftActivityJetNjets5', 'SubJet', 'fixedGridRhoFastjetCentralChargedPileUp', 'fixedGridRhoFastjetCentral', 'Electron', 'fixedGridRhoFastjetAll', 'GenPart', 'fixedGridRhoFastjetCentralCalo', 'boostedTau', 'GenJet', 'FsrP

['nCorrT1METJet',
 'CorrT1METJet_area',
 'CorrT1METJet_eta',
 'CorrT1METJet_muonSubtrFactor',
 'CorrT1METJet_phi',
 'CorrT1METJet_rawPt',
 'nFatJet',
 'FatJet_area',
 'FatJet_btagCSVV2',
 'FatJet_btagDDBvLV2',
 'FatJet_btagDDCvBV2',
 'FatJet_btagDDCvLV2',
 'FatJet_btagDeepB',
 'FatJet_btagHbb',
 'FatJet_deepTagMD_H4qvsQCD',
 'FatJet_deepTagMD_HbbvsQCD',
 'FatJet_deepTagMD_TvsQCD',
 'FatJet_deepTagMD_WvsQCD',
 'FatJet_deepTagMD_ZHbbvsQCD',
 'FatJet_deepTagMD_ZHccvsQCD',
 'FatJet_deepTagMD_ZbbvsQCD',
 'FatJet_deepTagMD_ZvsQCD',
 'FatJet_deepTagMD_bbvsLight',
 'FatJet_deepTagMD_ccvsLight',
 'FatJet_deepTag_H',
 'FatJet_deepTag_QCD',
 'FatJet_deepTag_QCDothers',
 'FatJet_deepTag_TvsQCD',
 'FatJet_deepTag_WvsQCD',
 'FatJet_deepTag_ZvsQCD',
 'FatJet_eta',
 'FatJet_mass',
 'FatJet_msoftdrop',
 'FatJet_n2b1',
 'FatJet_n3b1',
 'FatJet_particleNetMD_QCD',
 'FatJet_particleNetMD_Xbb',
 'FatJet_particleNetMD_Xcc',
 'FatJet_particleNetMD_Xqq',
 'FatJet_particleNet_H4qvsQCD',
 'FatJet_particleNet_Hb

In [2]:
#need to remove use of coffea hist
from coffea import processor, hist

##### TO DO #####
#make mass vs pt and response matrix (pt_gen, mass_gen, pt_reco, mass_reco)
# Add eta/phi/delta_r/pt cuts fully
# Make 2 eta collections --> high eta (>1.7) and central (<1.7)
# add same cuts on GenJetAK8
# find misses --> need to do deltaR matching by hand --> if no reco miss
# do Rivet routine
# make central (eta < 1.7) and high eta bins (1.7 < eta < 2.5)
# try AK4 jets to give low pT ??
# remove phi cuts --  why past me why?? do you mean try with and without phi cuts?

#bcut options: b_loose (apply loose bTag threshold to only hardest jet), bb_loose (apply loose bTag to leading two jets),
#              b_med(apply medium bTag to only the hardest jet), bb_med (apply medium bTag to leading two jets)

bTag_options = ['bbloose', 'bloose', 'bbmed', 'bmed']
def applyBTag(events, btag):
    print('btag input: ', btag, '\n')
    if (btag == 'bbloose'):
        events = events[(events.FatJet[:,0].btagCSVV2 >= 0.460) & (events.FatJet[:,1].btagCSVV2 >= 0.460)]
        print('Loose WP CSV V2 B tag applied to leading two jets')
    elif (btag == 'bloose'):
        events = events[(events.FatJet[:,0].btagCSVV2 >= 0.460)]
        print('Loose WP CSV V2 B tag applied to leading jet only')
    elif (btag == 'bbmed'):
        events = events[(events.FatJet[:,0].btagCSVV2 >= 0.80) & (events.FatJet[:,1].btagCSVV2 >= 0.80)]
        print('Medium WP CSV V2 B tag applied to first two jets')
    elif (btag == 'bmed'):
        events = events[(events.FatJet[:,0].btagCSVV2 >= 0.80)]
        print('Medium WP CSV V2 B tag applied to leading jet only')
    else:
        # printing below when it shoudln't be
        events = events
        print('no btag applied')
    return events

class TriJetHists(processor.ProcessorABC):
    def __init__(self, ptCut = 200., etaCut = 2.4, btag = 'null'):
        self.ptCut = ptCut
        self.etaCut = etaCut
        self.btag = btag
        dataset_cat = hist.Cat("dataset", "Dataset")
        jet_cat = hist.Cat("jetNumb", "Jet")
        parton_cat = hist.Cat("partonFlav", "Parton Flavour")
        mass_bins = hist.Bin("mass", "Jet Mass (GeV)", 60, 0, 200)
        pt_bins = hist.Bin("pt", "Jet pT (GeV)", 60, 0, 2400)
        disc_bins = hist.Bin("bdisc", "B-tag discriminator", 10, 0.0, 1.)
        eta_bins = hist.Bin("eta", "Eta", 10, -2.5, 2.5)
        self._histos = processor.dict_accumulator({
        'jet_pt':          hist.Hist("Events", jet_cat, parton_cat, pt_bins),
        'jet_eta':         hist.Hist("Events", jet_cat, parton_cat, eta_bins),
        'jet_mass':        hist.Hist("Events", jet_cat, parton_cat, mass_bins),
        'genjet_pt':       hist.Hist("Events", jet_cat, pt_bins),
        'genjet_eta':      hist.Hist("Events", jet_cat, eta_bins),
        'genjet_mass':     hist.Hist("Events", jet_cat, mass_bins),
        'weights':         hist.Hist("Events", dataset_cat, hist.Bin("weights", "gen weights", 60, 0, 1), ),
        'btag':            hist.Hist("Events", jet_cat, parton_cat, disc_bins),
        'gluonPurity':     hist.Hist("Events", parton_cat, disc_bins),
        'nGluonJets':      hist.Hist("Events", jet_cat, ),
        'cutflow':         processor.defaultdict_accumulator(int),
        })
    
    @property
    def accumulator(self):
        return self._histos
    def process(self, events):
        out = self.accumulator.identity()
        trijetEvents = events[
            (ak.num(events.FatJet) >= 3) & (ak.num(events.GenJetAK8) >= 3)
        ]
        #get leading 3 jets
        print("Initial number of matched trijet events after all cuts ", len(trijetEvents), '\n')
        jet1 = trijetEvents.FatJet[:, 0]
        jet2 = trijetEvents.FatJet[:, 1]
        jet3 = trijetEvents.FatJet[:, 2]
        
#        print("Softest jets after >3 selection", len(jet3))
        
        
        #calculate dphi_min
        dphi12 = np.abs(jet1.delta_phi(jet2))
        dphi13 = np.abs(jet1.delta_phi(jet3))
        dphi23 = np.abs(jet2.delta_phi(jet3))
        
        dphi_min = np.amin([dphi12, dphi13, dphi23], axis = 0)
        
#         print("Fields available for FatJets ", trijetEvents.FatJet.fields)
#         print("Fields available for GenJets ", trijetEvents.GenJetAK8.fields)
        
        #do same for gen; jets might not be same order reco and gen
        genjet1 = trijetEvents.GenJetAK8[:,0]
        genjet2 = trijetEvents.GenJetAK8[:,1]
        genjet3 = trijetEvents.GenJetAK8[:,2]
        
        dphi12_gen = np.abs(genjet1.delta_phi(genjet2))
        dphi13_gen = np.abs(genjet1.delta_phi(genjet3))
        dphi23_gen = np.abs(genjet2.delta_phi(genjet3))
        
        dphimin_gen = np.amin([dphi12_gen, dphi13_gen, dphi23_gen], axis = 0)
        
        #revisit pt cut at some point?
        pt_cut = (ak.all(trijetEvents.FatJet.pt > self.ptCut, axis = -1) &
                  ak.all(trijetEvents.GenJetAK8.pt > self.ptCut, axis = -1))
        eta_cut = (ak.all(np.abs(trijetEvents.FatJet.eta) < self.etaCut, axis = -1) &
                   ak.all(np.abs(trijetEvents.GenJetAK8.eta) < self.etaCut, axis = -1))
        asymm = np.abs(jet1.pt - jet2.pt)/(jet1.pt + jet2.pt)
        asymm_gen  = np.abs(genjet1.pt - genjet2.pt)/(genjet1.pt + genjet2.pt)
        
        #apply dphi gen and reco selection, eta, and pt cut
        #selections are based on arXiv:1807.05974v2

        trijetEvents = trijetEvents[(dphimin_gen > 1.0) & (dphi_min > 1.0) & (pt_cut) & (eta_cut) &
                                    (asymm < 0.3) & (asymm_gen < 0.3)
                                    ]
        print(trijetEvents)
        #make matching mask - where there is any none in FatJet give False value
        matched = ~ak.any(ak.is_none(trijetEvents.FatJet.matched_gen, axis = -1), axis = -1)
        # NEED TO MAKE DELTA R MATCHING FOR GEN TO FIND MISSES
        #gen_matched = ~ak.is_none(DNE)
        
        
        # fakes are events reconstructed but does not exist in MC
        fakes = trijetEvents.FatJet[ak.any(ak.is_none(trijetEvents.FatJet.matched_gen, axis = -1), axis = -1)]
        print("Number of fake jets ", len(fakes))
        
        #miss = jet in MC but not reconstructed
        #misses = genJets[ak.any(ak.is_none(DNE), axis = -1)]
        
        trijetEvents = trijetEvents[matched]
        print("Number of matched trijet events after matching ", len(trijetEvents), '\n')
        
        
        #apply bTag
        trijetEvents = applyBTag(trijetEvents, self.btag)
        print("Number of matched trijet events after all cuts ", len(trijetEvents), '\n')
        
        
        jet1 = trijetEvents.FatJet[:,0]
        jet2 = trijetEvents.FatJet[:,1]
        jet3 = trijetEvents.FatJet[:,2]
        
        genjet1 = jet1.matched_gen
        genjet2 = jet2.matched_gen
        genjet3 = jet3.matched_gen
        

#        print("Softest jets after dphi selection and matching", len(jet3))
        
        #flavour --> 21 is gluon
        jet3_g     = jet3[np.abs(genjet3.partonFlavour) == 21]
        jet3_uds   = jet3[np.abs(genjet3.partonFlavour) < 4]
        jet3_c     = jet3[np.abs(genjet3.partonFlavour) == 4]
        jet3_b     = jet3[np.abs(genjet3.partonFlavour) == 5]
        jet3_other = jet3[(np.abs(genjet3.partonFlavour) > 5) & (np.abs(genjet3.partonFlavour) != 21)]
        
        jet2_g     = jet2[np.abs(genjet2.partonFlavour) == 21]
        jet2_uds   = jet2[np.abs(genjet2.partonFlavour) < 4]
        jet2_c     = jet2[np.abs(genjet2.partonFlavour) == 4]
        jet2_b     = jet2[np.abs(genjet2.partonFlavour) == 5]
        jet2_other = jet2[(np.abs(genjet2.partonFlavour) > 5) & (np.abs(genjet2.partonFlavour) != 21)]
        
        jet1_g     = jet1[np.abs(genjet1.partonFlavour) == 21]
        jet1_uds   = jet1[np.abs(genjet1.partonFlavour) < 4]
        jet1_c     = jet1[np.abs(genjet1.partonFlavour) == 4]
        jet1_b     = jet1[np.abs(genjet1.partonFlavour) == 5]
        jet1_other = jet1[(np.abs(genjet1.partonFlavour) > 5) & (np.abs(genjet1.partonFlavour) != 21)]
        
        print("Check for none values", ak.any(ak.is_none(jet3_g.mass), axis = -1))
        
        print("Gluon purity of third jet for basic selection: ", len(jet3_g)/len(ak.flatten(jet3)))
        
        
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "Gluon",  mass = jet1_g.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "UDS",    mass = jet1_uds.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "Charm",  mass = jet1_c.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "Bottom", mass = jet1_b.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "Other",  mass = jet1_other.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "Gluon",  mass = jet2_g.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "UDS",    mass = jet2_uds.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "Charm",  mass = jet2_c.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "Bottom", mass = jet2_b.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "Other",  mass = jet2_other.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet3", partonFlav = "Gluon",  mass = jet3_g.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet3", partonFlav = "UDS",    mass = jet3_uds.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet3", partonFlav = "Charm",  mass = jet3_c.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet3", partonFlav = "Bottom", mass = jet3_b.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet3", partonFlav = "Other",  mass = jet3_other.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "Gluon",  pt = jet1_g.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "UDS",    pt = jet1_uds.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "Charm",  pt = jet1_c.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "Bottom", pt = jet1_b.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "Other",  pt = jet1_other.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "Gluon",  pt = jet2_g.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "UDS",    pt = jet2_uds.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "Charm",  pt = jet2_c.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "Bottom", pt = jet2_b.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "Other",  pt = jet2_other.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet3", partonFlav = "Gluon",  pt = jet3_g.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet3", partonFlav = "UDS",    pt = jet3_uds.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet3", partonFlav = "Charm",  pt = jet3_c.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet3", partonFlav = "Bottom", pt = jet3_b.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet3", partonFlav = "Other",  pt = jet3_other.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "Gluon",  eta = jet1_g.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "UDS",    eta = jet1_uds.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "Charm",  eta = jet1_c.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "Bottom", eta = jet1_b.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "Other",  eta = jet1_other.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "Gluon",  eta = jet2_g.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "UDS",    eta = jet2_uds.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "Charm",  eta = jet2_c.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "Bottom", eta = jet2_b.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "Other",  eta = jet2_other.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet3", partonFlav = "Gluon",  eta = jet3_g.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet3", partonFlav = "UDS",    eta = jet3_uds.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet3", partonFlav = "Charm",  eta = jet3_c.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet3", partonFlav = "Bottom", eta = jet3_b.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet3", partonFlav = "Other",  eta = jet3_other.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        #NOTE --> need gen sd mass eventually --> recluster :( 
        out['genjet_mass'].fill(
            jetNumb = "genjet1",
            mass=genjet1.mass,
            #weight=trijetEvents.Generator.weight[~ak.is_none(trijetEvents.FatJet[:,0].matched_gen)]
        )
        out['genjet_mass'].fill(
            jetNumb = "genjet2",
            mass=genjet2.mass,
            #weight=trijetEvents.Generator.weight[matched2]
        )
        out['genjet_mass'].fill(
            jetNumb = "genjet3",
            mass=genjet3.mass,
            #weight=trijetEvents.Generator.weight[matched3]
        )
        out['weights'].fill(
            dataset=events.metadata["dataset"],
            weights=trijetEvents.Generator.weight,
        )     
        out['btag'].fill(jetNumb = "jet1", partonFlav = "Gluon",  bdisc = jet1_g.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet1", partonFlav = "UDS",    bdisc = jet1_uds.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet1", partonFlav = "Charm",  bdisc = jet1_c.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet1", partonFlav = "Bottom", bdisc = jet1_b.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet1", partonFlav = "Other",  bdisc = jet1_other.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet2", partonFlav = "Gluon",  bdisc = jet2_g.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet2", partonFlav = "UDS",    bdisc = jet2_uds.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet2", partonFlav = "Charm",  bdisc = jet2_c.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet2", partonFlav = "Bottom", bdisc = jet2_b.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet2", partonFlav = "Other",  bdisc = jet2_other.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet3", partonFlav = "Gluon",  bdisc = jet3_g.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet3", partonFlav = "UDS",    bdisc = jet3_uds.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet3", partonFlav = "Charm",  bdisc = jet3_c.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet3", partonFlav = "Bottom", bdisc = jet3_b.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['btag'].fill(jetNumb = "jet3", partonFlav = "Other",  bdisc = jet3_other.btagCSVV2,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['cutflow']['chunks'] += 1
        out['cutflow']['nGluonJets'] += (len(jet3[np.abs(genjet3.partonFlavour) == 21]))
        return out
    
    def postprocess(self, accumulator):
        return accumulator

/opt/conda/lib/python3.8/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [3]:
tstart = time.time()
samples = {
    "QCD": files
}

result = processor.run_uproot_job(
    samples,
    "Events",
    TriJetHists(),
    processor.futures_executor,
    {"schema": NanoAODSchema},
)
elapsed = time.time() - tstart
print(result)
print("Time taken to run over samples ", elapsed)
util.save(result, 'coffeaOutput/TriJet_pT200_eta24_nobtag_result.coffea')

### Need to move everything below this to a separate script that reads the .coffea output to save time

KeyError: 'files'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


numerator = result['jet_pt'].integrate('partonFlav','Gluon').sum('jetNumb')
denominator = result['jet_pt'].sum('partonFlav').sum('jetNumb')
print(numerator, denominator)

numerator.label = r'$\epsilon$'
ax = hist.plotratio(
    num=numerator,
    denom=denominator,
    error_opts={'color': 'k', 'marker': '.'},
    unc='clopper-pearson'
)

glu = result['jet_pt'].sum('jetNumb').values()[('Gluon',)]
tot_parton = result['jet_pt'].sum('jetNumb').sum('partonFlav').values()[()]
print(glu)
print(tot_parton)

glu_frac = np.where(tot_parton > 0., np.divide(glu,tot_parton), 0)

print(len(result['jet_pt'].axis('pt').centers()))
print(result['jet_pt'].to_hist())

print(glu_frac)
plt.scatter(result['jet_pt'].axis('pt').centers(), glu_frac)

plt.xlabel("Pt (GeV)")
plt.ylabel("Gluon fraction")
plt.ylim([0.,1.0])

In [ ]:
numerator = result['jet_eta'].integrate('partonFlav','Gluon').sum('jetNumb')
denominator = result['jet_eta'].sum('partonFlav').sum('jetNumb')
print(numerator, denominator)

numerator.label = r'$\epsilon$'
ax = hist.plotratio(
    num=numerator,
    denom=denominator,
    error_opts={'color': 'k', 'marker': '.'},
    unc='clopper-pearson'
)

plt.xlabel("Eta < 2.4")
plt.ylabel("Gluon fraction")
plt.ylim([0.,1.0])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (20,5)


# fig, axs = plt.subplots(1, 3)
print(result['jet_mass'].sum('jetNumb'))
hist.plot1d(result['jet_mass'].sum('jetNumb'), stack = True)
#axs[0].set_xlabel("Jet mass (GeV)")
plt.xlabel("Jet mass (GeV)")

In [ ]:
# plot mass after summing over partons

In [ ]:
fig, ax = plt.subplots(1, 1)
hist.plot1d(result['jet_pt'].integrate('jetNumb', 'jet3'), stack = True)
ax.set_xlabel("Third Jet pT (GeV)")
ax.set_xscale("log")
ax.set_xlim([180., 2000.])

In [ ]:
fig, ax = plt.subplots(1, 1)
hist.plot1d(result['btag'].integrate('jetNumb', 'jet3'), stack = True)
ax.set_xlabel("Jet3 b-tag disc.")
#can do an anti b-tag --> inverse of recommendation

In [ ]:
print(result['cutflow']['nGluonJets'])

#plot pt, eta, phi, fake rate, pt efficiency, gluon purity for difference selections with b tags for all jets 
#and each individual jet in the 3 jets